In [4]:
import numpy as np
from scipy.optimize import curve_fit    # フィッティング用
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
import os
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import time
import csv
#訓練データの作成を早くすることを目的とした．
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE' #意味はわからん

In [6]:
start = time.time()
train_num = 10000 #訓練データの数
test_num = 1000 #テストデータの数
data_size = 100 #配列の大きさ

x_train = np.zeros((train_num,data_size))
y_train = np.zeros((train_num,data_size))
tx_train = np.zeros((train_num, 3))
ty_train = np.zeros((train_num, 3))
t_1_train = np.zeros(train_num)
x_test = np.zeros((test_num,data_size))
y_test = np.zeros((test_num,data_size))
tx_test = np.zeros((test_num, 3))
ty_test = np.zeros((test_num, 3))
t_1_test = np.zeros(test_num)

# ガウシアンビームの関数の定義
def gaussian_beam(x,a,b,c,d):
    return  a * np.exp(-2*(x-b)*(x-b)/c/c) + d

# ガウシアンビームのパラメータ
i0 = 1.0
x0 = np.arange(0.0, 2.0, 0.1)
y0 = np.arange(0.0, 2.0, 0.1)
w0 = 5.0
h0 = 0.0

# 信号に対するノイズ量[%]
NOISE = 30


def create_data(N, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N,i0, x0, y0, w0, h0, x, y, tx, ty, t1):
    n_N      = np.arange(N) +1
    x0_N     = np.zeros(N)   # N回分のx0,w0を格納する配列
    y0_N     = np.zeros(N)
    w0_x_N   = np.zeros(N)
    w0_y_N   = np.zeros(N)
    h0_x_N   = np.zeros(N)
    h0_y_N   = np.zeros(N)
    
    x0 = x0.repeat(N/20) #20stepの中心値
    y0 = y0.repeat(N/20)
    
    # x配列とy配列
    x_array = np.arange(-50, 50, 1.0)                         # x配列
    y_array = np.arange(-50, 50, 1.0)                         # y配列
    nx = len(x_array)
    ny = len(y_array)
    intensity = np.zeros((N, nx, ny))                            # ノイズを含まない2次元強度分布
    for i in range(nx):
        for j in range(ny):
            intensity[:,i,j] = i0 * np.exp(-2*((x_array[i]-x0[i])*(x_array[i]-x0[i]) + (y_array[j]-y0[j])*(y_array[j]-y0[j]))/w0/w0)
            
    # 2次元の強度分布にノイズを付与
    intensity_noise = np.zeros((N, nx, ny))
    noise = (np.random.rand(N*nx*ny)-0.5)*i0*NOISE*0.01   # プラスマイナスNOISE%のノイズ(一様分布), (np.random.rand(nx*ny)-0.5)*2の部分が-1から1までの乱数になる
    noise = noise.reshape((N,nx,ny))
    intensity_noise = intensity + noise
    
    # 最大強度を取る位置における強度プロファイル
    profile_x = np.zeros(nx)
    profile_y = np.zeros(ny)
    idx = intensity_noise.max(axis=1).argmax(axis=1)
    idy = intensity_noise.max(axis=2).argmax(axis=1)
    
    for n in range(N):
        x[n] = intensity_noise[n,:,idy[n]]
        y[n] = intensity_noise[n,idx[n],:]
        tx[n] = (i0, x0[n], w0)
        ty[n] = (i0, y0[n], w0)
        t1[n] = x0[n]
        
        param_ini_x = np.array([i0, x0[n], w0, h0])  # フィッティングの初期値 N×4 (ここではデータから推定は行わない)
        param_ini_y = np.array([i0, y0[n], w0, h0])
        # x方向のプロファイルの非線形フィッティング
        param, cov  = curve_fit(gaussian_beam, x_array, profile_x, p0=param_ini_x, maxfev=2000)
        i0_x        = param[0]
        x0_N[n]     = param[1]
        w0_x_N[n]   = param[2]
        h0_x_N[n]   = param[3]
        # y方向のプロファイルの非線形フィッティング
        param, cov  = curve_fit(gaussian_beam, y_array, profile_y, p0=param_ini_y, maxfev=2000)
        i0_y        = param[0]
        y0_N[n]     = param[1]
        w0_y_N[n]   = param[2]
        h0_y_N[n]   = param[3]

    return N, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, x, y, tx, ty

# N回繰り返しフィッティングを行う
N        = 10000
n_N      = np.zeros(N)
x0_N     = np.zeros(N)   # N回分のx0,w0を格納する配列
y0_N     = np.zeros(N)
w0_x_N   = np.zeros(N)
w0_y_N   = np.zeros(N)
h0_x_N   = np.zeros(N)
h0_y_N   = np.zeros(N)

create_data(train_num, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, 
            i0, x0, y0, w0, h0, x_train, y_train, tx_train, ty_train, t_1_train)

# N回繰り返しフィッティングを行う
N        = 1000
n_N      = np.zeros(N)
x0_N     = np.zeros(N)   # N回分のx0,w0を格納する配列
y0_N     = np.zeros(N)
w0_x_N   = np.zeros(N)
w0_y_N   = np.zeros(N)
h0_x_N   = np.zeros(N)
h0_y_N   = np.zeros(N)

create_data(test_num, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, 
            i0, x0, y0, w0, h0, x_test, y_test, tx_test, ty_test, t_1_test)

t_train = t_1_train
t_test = t_1_test


elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:20.118541955947876[sec]


In [7]:
#訓練データ,テストデータの保存
save_x_train = "/Users/nagaiyuma/Desktop/maindata/x_train.csv"
save_t_train = "/Users/nagaiyuma/Desktop/maindata/t_train.csv"
save_x_test = "/Users/nagaiyuma/Desktop/maindata/x_test.csv"
save_t_test = "/Users/nagaiyuma/Desktop/maindata/t_test.csv"

with open(save_x_train,'w') as f:
    writer = csv.writer(f)
    writer.writerows(x_train)
with open(save_t_train,'w') as f:
    writer = csv.writer(f)
    writer.writerow(t_train)
with open(save_x_test,'w') as f:
    writer = csv.writer(f)
    writer.writerows(x_test)
with open(save_t_test,'w') as f:
    writer = csv.writer(f)
    writer.writerow(t_test)

In [20]:
#CNNの実装
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv1D, MaxPooling1D

batch_size = 200  # 訓練データを200ずつのデータに分けて学習させる
epochs = 50 # 訓練データを繰り返し学習させる数

#データ形式の変更(シーケンス長, パラメータ数)
train_x = x_train.reshape(-1, 100, 1)
train_t = tx_train
test_x = x_test.reshape(-1, 100, 1)
test_t = tx_test

model = Sequential()
model.add(Conv1D(50, 3, padding='same', input_shape=(100, 1), activation='relu', kernel_initializer="he_nomal"))
model.add(MaxPooling1D(2, padding='same'))
model.add(Conv1D(50, 3, padding='same', activation='relu', kernel_initializer="he_nomal"))
model.add(MaxPooling1D(5, padding='same'))
model.add(Conv1D(10, 3, padding='same', activation='relu', kernel_initializer="he_nomal"))
model.add(layers.Flatten())
model.add(Dense(100, activation='relu', kernel_initializer="he_nomal"))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu', kernel_initializer="he_nomal"))
model.add(Dropout(0.2))
model.add(Dense(3, activation='relu', kernel_initializer="he_nomal"))

model.summary()

model.compile(loss='mse',
 optimizer='adam',
 metrics=['mae'])

history = model.fit(train_x, train_t,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_data=(test_x, test_t))

score = model.evaluate(test_x, test_t, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 100, 50)           200       
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 50, 50)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 50, 50)            7550      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 10, 50)            0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 10, 10)            1510      
_________________________________________________________________
flatten_3 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)              

KeyboardInterrupt: 

In [13]:
#modelの保存
save_model_path = "/Users/nagaiyuma/Desktop/maindata/model_cnn.h5"
model.save(save_model_path)

In [16]:
#学習パラメータの取得
save_model_path = "/Users/nagaiyuma/Desktop/maindata/model_cnn.h5"
model = load_model(save_model_path)

#predict_x0の確認
print(model.predict(test_x).reshape(1000,3).T[0])
print(model.predict(test_x).reshape(1000,3).T[1])
print(model.predict(test_x).reshape(1000,3).T[2])

[1.0952404 1.0981549 1.0905627 1.0918893 1.0983261 1.0943263 1.0901543
 1.0934758 1.092184  1.092725  1.1009378 1.0890833 1.0947455 1.0914627
 1.0833505 1.102924  1.1064401 1.0930467 1.0901797 1.0917666 1.0921512
 1.0966629 1.090691  1.0928748 1.0952435 1.0939738 1.0880044 1.0940893
 1.0894383 1.1034738 1.0946383 1.0928923 1.0917332 1.0930401 1.0990136
 1.0974483 1.0934081 1.0942196 1.0808322 1.0893888 1.0902541 1.088207
 1.1039424 1.0944775 1.0921052 1.0887935 1.0913438 1.0914543 1.1013227
 1.0938206 1.0991504 1.0935855 1.0953867 1.0907693 1.092255  1.088342
 1.0926437 1.0996927 1.0862145 1.0919836 1.0974971 1.1003898 1.0949007
 1.0846772 1.0902739 1.1040245 1.0885185 1.0894084 1.0897561 1.087484
 1.1016241 1.1029551 1.0842553 1.0863824 1.0899699 1.0940529 1.1016811
 1.093326  1.0973867 1.0827389 1.093554  1.0973713 1.1003907 1.0792155
 1.0892644 1.0913479 1.0881565 1.0887511 1.097034  1.0892444 1.0902773
 1.0967995 1.0909938 1.0894047 1.0897228 1.0958344 1.0998472 1.0977397
 1.095464

[1.0776807 1.0562396 1.0709846 1.0747758 1.0594249 1.0638448 1.0694392
 1.0673146 1.0554793 1.0806065 1.0757731 1.0563568 1.0654907 1.0559552
 1.0392295 1.0855463 1.0632952 1.0821278 1.0688456 1.0575495 1.0753996
 1.0406826 1.0788616 1.0716242 1.0458146 1.054126  1.0630984 1.0736521
 1.0684865 1.0890788 1.0461212 1.0544368 1.0899209 1.0584177 1.0435611
 1.0602684 1.0772271 1.0608697 1.05227   1.044356  1.0740852 1.0469285
 1.0494293 1.0679476 1.0816464 1.0757308 1.0773044 1.0572855 1.0659474
 1.0945218 1.0526652 1.0618229 1.0482908 1.0442268 1.060491  1.071947
 1.061386  1.0667737 1.0601712 1.0662309 1.059475  1.0758512 1.0549481
 1.0927768 1.0756872 1.0774115 1.0587314 1.0613596 1.0783077 1.0707403
 1.0714321 1.0761348 1.0471958 1.072853  1.0620315 1.0590316 1.0635751
 1.0562791 1.0893364 1.0491595 1.0764222 1.0721862 1.0544927 1.0551261
 1.0612748 1.0784013 1.0752277 1.072279  1.0840732 1.0544962 1.0381606
 1.056824  1.0411514 1.0692651 1.076923  1.0764621 1.0615808 1.061653
 1.06409

[4.6442213 4.623729  4.686319  4.6163235 4.655056  4.674011  4.637671
 4.689971  4.6621027 4.6324863 4.666173  4.641423  4.6316876 4.6347294
 4.642106  4.6500964 4.6505213 4.6512403 4.6526766 4.6189194 4.6761622
 4.629012  4.6769295 4.6643257 4.6639624 4.6564984 4.681378  4.6607943
 4.6014104 4.6431184 4.685634  4.6280584 4.627391  4.65817   4.664446
 4.6684046 4.654108  4.6175117 4.641708  4.6347446 4.6553965 4.645027
 4.6212964 4.704562  4.624331  4.658491  4.6186576 4.673847  4.6502876
 4.5920706 4.6536107 4.665118  4.643426  4.685197  4.632352  4.6719604
 4.660183  4.691877  4.649454  4.6615124 4.642329  4.6086445 4.650814
 4.651055  4.6473064 4.681642  4.603773  4.673092  4.691448  4.658743
 4.5951076 4.6373196 4.5968246 4.6673746 4.6361523 4.630529  4.6500297
 4.632832  4.660911  4.6307306 4.622882  4.646167  4.6483603 4.66522
 4.6246495 4.6127825 4.6538486 4.657163  4.648596  4.6517105 4.6304073
 4.6247377 4.662838  4.6551247 4.6441383 4.586515  4.589794  4.6223636
 4.68645   4.

In [18]:
abs_error = abs(np.array(model.predict(test_x).reshape(1000,3)).T[0]- tx_test.T[0])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)
abs_error = abs(np.array(model.predict(test_x).reshape(1000,3)).T[1]- tx_test.T[1])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)
abs_error = abs(np.array(model.predict(test_x).reshape(1000,3)).T[2]- tx_test.T[2])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)

0.09284683001041412 0.006017291430585471
0.5068420162916185 0.3007633356328077
0.34991920471191407 0.026563036661073552


In [6]:
batch_size = 200  # 訓練データを128ずつのデータに分けて学習させる
epochs = 50 # 訓練データを繰り返し学習させる数

# モデルの作成
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(100,)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3))

model.summary()

model.compile(loss='mse',
 optimizer=RMSprop(),
 metrics=['mae'])

# 学習は、scrkit-learnと同様fitで記述できる
history = model.fit(x_train, tx_train,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_data=(x_test, tx_test))

# 評価はevaluateで行う
score = model.evaluate(x_test, tx_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 20,503
Trainable params: 20,503
Non-trainable params: 0
_________________________________________________________________
Train on 10000 samples, validate on 1000 samples
Epoch 1/50
10000/10000 [==============================] - 2s 167us/

In [7]:
#modelの保存
save_model_path = "/Users/nagaiyuma/Desktop/maindata/model_3.h5"
model.save(save_model_path)

In [8]:
abs_error = abs(np.array(model.predict(x_test).reshape(1000,3)).T[1]- tx_test.T[1])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)

0.5006622524619104 0.28810336643133316
